### Import libraries ###

In [1]:
import os
import pandas as pd
import librosa
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tqdm import tqdm
import pickle
import re
from IPython.display import Audio
from sklearn.metrics.pairwise import cosine_similarity
import random

2025-07-10 10:15:44.154143: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752142544.437949      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752142544.538422      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Load VGGish model ###

In [2]:
vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
print("VGGish model loaded successfully.")

2025-07-10 10:16:13.759864: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


VGGish model loaded successfully.


### Load the CSV file ###

In [3]:
csv_path = '/kaggle/input/spotify-musics/music_info.csv'
df = pd.read_csv(csv_path)

### Create dictionary for mapping track_id to spotify_id ###

In [4]:
trackid_to_spotifyid = dict(zip(df['track_id'], df['spotify_id']))
print(f'Length: {len(trackid_to_spotifyid)}')

print(f'First 5 elements in dictionary:')
print(f'     track_id           spotify_id')
for i, (key, value) in enumerate(trackid_to_spotifyid.items()):
    if i >= 5:
        break
    print(key, value)

# for x in trackid_to_spotifyid:
#     print(x, trackid_to_spotifyid[x])

Length: 50683
First 5 elements in dictionary:
     track_id           spotify_id
TRIOREW128F424EAF0 09ZQ5TmUG8TSL56n0knqrj
TRRIVDJ128F429B0E8 06UfBBDISthj1ZJAtX4xjj
TROUVHL128F426C441 0keNu0t0tqsWtExGM3nT1D
TRUEIND128F93038C4 0ancVQ9wEcHVd0RrGICTE4
TRLNZBD128F935E4D8 01QoK9DA7VTeTSE3MNzp4I


### Get list of MP3 files from tracks folder ###

In [5]:
tracks_folder = '/kaggle/input/spotify-musics/tracks/tracks'
mp3_files = [f for f in os.listdir(tracks_folder) if f.endswith('.mp3')]
print(len(mp3_files))
print(mp3_files[0])
print(mp3_files[-1])

1500
Electronic-TRVBGXF128F1463589.mp3
Country-TRUTNVB128F92F3682.mp3


### Define functions to preprocess audio files and extract embeddings ###

In [6]:
def preprocess_audio(file_path):
    """
    Load and preprocess an MP3 file for VGGish.
    Returns audio resampled to 16kHz, mono.
    """
    try:
        # Load audio file with librosa
        audio, sr = librosa.load(file_path, sr=16000, mono=True)
        # Ensure audio is at least 0.96 seconds (VGGish frame length)
        if len(audio) < 0.96 * sr:
            audio = np.pad(audio, (0, int(0.96 * sr) - len(audio)), mode='constant')
        return audio
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def extract_vggish_embedding(audio):
    """
    Extract VGGish embedding from preprocessed audio.
    Aggregates frame-level embeddings by averaging.
    """
    try:
        # VGGish expects input in shape (num_samples,)
        embeddings = vggish_model(audio)
        # Mean pooling over time to get a single 128-dimensional embedding
        embedding = np.mean(embeddings, axis=0)
        return embedding
    except Exception as e:
        print(f"Error extracting embedding: {e}")
        return None

### Process a sample mp3 file and extract the corresponding embedding ###

In [7]:
temp_arr = []

sample_file = mp3_files[0]
match = re.match(r"[^-]+\-(.+)\.mp3", sample_file)
if not match:
    print(f"Invalid filename format: {sample_file}")
track_id = match.group(1)
spotify_id = trackid_to_spotifyid.get(track_id)
if not spotify_id:
    print(f"No spotify_id found for track_id: {track_id}")

# Load and preprocess .mp3 audio file
file_path = os.path.join(tracks_folder, sample_file)
audio = preprocess_audio(file_path)
if audio is None:
    print("Audio is None!")

# Extract embedding
embedding = extract_vggish_embedding(audio)
if embedding is None:
     print("Embedding is None!")

temp_arr.append((spotify_id, embedding))
print(f'Play track:')
audio_widget = Audio(file_path, autoplay=False)
display(audio_widget)
print(f'spotify_id: {temp_arr[0][0]}')
print(f'Length of embedding: {len(temp_arr[0][1])}')
print(f'Embedding: {temp_arr[0][1]}')

Play track:


spotify_id: 0flZwJevfk2DptAn9HMCry
Length of embedding: 128
Embedding: [-0.6881859   0.13197668  0.24371165 -0.32256004 -0.06723153 -0.5038437
  0.16905947  0.1745735   0.02652082 -0.4127807   0.28173277 -0.32677862
 -0.2594389  -0.1983168  -0.17495076 -0.01649017  0.25907     0.11717893
 -0.4159799  -0.32746384 -0.16823085 -0.26254117  0.07047758 -0.7042922
  0.13725783  0.06062803  0.06479052 -0.11237985 -0.0372535  -0.15076794
 -0.06765186  0.34830928  0.08438249  0.08398769  0.24307765 -0.18607523
  0.09483907 -0.3997182  -0.25020593 -0.19169664  0.37169242 -0.55265725
 -0.1614121  -0.0920088   0.0368267  -0.24559091  0.3641013   0.24568455
 -0.35796806  0.4000602   0.16157448 -0.38644803 -0.9282457   0.06200619
  0.21786253 -0.28992468  0.40019426  0.2836535   0.08914917 -0.08305687
  0.23344024 -0.01339755  0.11852762 -0.16985223  0.08948165 -0.20980161
 -0.10880082 -0.09219437  0.33692655 -0.28639424  0.09030877  0.29244912
 -0.15783007 -0.33223155  0.09099139 -0.1063601  -0.625

### Process 1500 mp3 files and extract the corresponding embeddings ###

In [8]:
results = []

for mp3_file in tqdm(mp3_files, desc="Processing MP3 files"):
    # Extract track_id from filename (e.g., Blues-TRACOHF128F1498509.mp3)
    match = re.match(r"[^-]+\-(.+)\.mp3", mp3_file)
    if not match:
        print(f"Invalid filename format: {mp3_file}")
        continue
    track_id = match.group(1)
    spotify_id = trackid_to_spotifyid.get(track_id)
    if not spotify_id:
        print(f"No spotify_id found for track_id: {track_id}")
        continue

    # Load and preprocess .mp3 audio file
    file_path = os.path.join(tracks_folder, mp3_file)
    audio = preprocess_audio(file_path)
    if audio is None:
        continue

    # Extract embedding
    embedding = extract_vggish_embedding(audio)
    if embedding is None:
        continue

    # Append to results
    results.append((spotify_id, embedding))

Processing MP3 files: 100%|██████████| 1500/1500 [14:22<00:00,  1.74it/s]


### Save results to a .pkl file ###

In [9]:
output_file = "tracks_embeddings.pkl"
try:
    with open(output_file, 'wb') as f:
        pickle.dump(results, f)
    print(f"Embeddings saved to {output_file}")
except Exception as e:
    print(f"Error saving embeddings: {e}")

Embeddings saved to tracks_embeddings.pkl


### Load the .pkl file ###

In [10]:
pkl_path = '/kaggle/input/tracks-embeddings/tracks_embeddings.pkl'
with open(pkl_path, 'rb') as file:
    tracks_embeddings = pickle.load(file)

### Checking loaded file ###

In [11]:
print('Type of tracks_embeddings: ', type(tracks_embeddings))
print('Length of tracks_embeddings: ', len(tracks_embeddings))
print(f'spotify_id: {tracks_embeddings[0][0]}')
print(f'Length of embedding: {len(tracks_embeddings[0][1])}')
print(f'Embedding: {tracks_embeddings[0][1]}')

Type of tracks_embeddings:  <class 'list'>
Length of tracks_embeddings:  1500
spotify_id: 0flZwJevfk2DptAn9HMCry
Length of embedding: 128
Embedding: [-0.68818575  0.13197674  0.24371189 -0.32256    -0.06723172 -0.5038438
  0.16905946  0.1745734   0.02652138 -0.41278073  0.2817328  -0.3267785
 -0.25943887 -0.19831687 -0.17495061 -0.01648985  0.25906953  0.11717907
 -0.41598013 -0.32746357 -0.16823071 -0.26254135  0.07047758 -0.7042922
  0.1372576   0.06062786  0.06479052 -0.11237999 -0.03725365 -0.15076816
 -0.06765191  0.3483088   0.08438241  0.08398774  0.24307759 -0.18607536
  0.09483893 -0.39971793 -0.25020593 -0.19169647  0.37169236 -0.5526572
 -0.16141169 -0.09200899  0.03682638 -0.24559082  0.36410147  0.24568465
 -0.35796794  0.4000603   0.16157433 -0.38644806 -0.92824584  0.06200597
  0.21786208 -0.2899246   0.40019453  0.28365323  0.08914898 -0.0830568
  0.23344028 -0.01339766  0.11852799 -0.16985223  0.08948139 -0.2098016
 -0.10880087 -0.09219439  0.33692613 -0.28639394  0.09

### Define functions to compute cosine similarity and retrieve top N most similar songs for each song ###

In [12]:
def compute_cosine_similarities(tracks_embeddings):
    """
    Compute pairwise cosine similarities for all embeddings.
    Returns similarity matrix and list of spotify_ids.
    """
    # Extract spotify_ids and embeddings
    spotify_ids = [spotify_id for spotify_id, _ in tracks_embeddings]
    embeddings = np.array([embedding for _, embedding in tracks_embeddings])
    
    # Compute cosine similarity matrix
    similarity_matrix = cosine_similarity(embeddings)
    
    return spotify_ids, similarity_matrix

def get_top_n_similar(spotify_ids, similarity_matrix, N, tracks_embeddings):
    """
    Retrieve the top N most similar spotify_ids for each song.
    Returns list of (spotify_id, embedding, similar_ids) tuples.
    """
    results = []
    for i, spotify_id in enumerate(tqdm(spotify_ids, desc="Finding top N similar songs")):
        # Get similarity scores for this song
        sim_scores = similarity_matrix[i]
        
        # Exclude self (set own similarity to -inf)
        sim_scores[i] = -np.inf
        
        # Get indices of top N similar songs
        top_n_indices = np.argsort(sim_scores)[-N:][::-1]  # Descending order
        
        # Ensure we have enough valid indices
        if len(top_n_indices) < N:
            print(f"Warning: Only {len(top_n_indices)} similar songs found for {spotify_id}.")
        
        # Get corresponding spotify_ids
        similar_ids = [spotify_ids[idx] for idx in top_n_indices]
        
        # Append result
        embedding = tracks_embeddings[i][1]  # Original embedding
        results.append((spotify_id, embedding, similar_ids))
    
    return results

### Compute cosine similarities ###

In [13]:
spotify_ids, similarity_matrix = compute_cosine_similarities(tracks_embeddings)
print('Length of spotify_ids: ', len(spotify_ids))
print('Dimensions of similarity matrix: ', similarity_matrix.shape)

Length of spotify_ids:  1500
Dimensions of similarity matrix:  (1500, 1500)


### Retrieve top N similar songs ###

In [14]:
# Number of most similar songs for each song to retrieve
N = 15
results = get_top_n_similar(spotify_ids, similarity_matrix, N, tracks_embeddings)

Finding top N similar songs: 100%|██████████| 1500/1500 [00:00<00:00, 8975.52it/s]


### Checking format of top N similar songs ###

In [15]:
print('Length of results: ', len(results))
print('Length of result for first song: ', len(results[0]))
print('The spotify_id of first song: ', results[0][0])
print('Embedding of first song: ', results[0][1])
print('Top N similar songs to first song: ', results[0][2])

Length of results:  1500
Length of result for first song:  3
The spotify_id of first song:  0flZwJevfk2DptAn9HMCry
Embedding of first song:  [-0.68818575  0.13197674  0.24371189 -0.32256    -0.06723172 -0.5038438
  0.16905946  0.1745734   0.02652138 -0.41278073  0.2817328  -0.3267785
 -0.25943887 -0.19831687 -0.17495061 -0.01648985  0.25906953  0.11717907
 -0.41598013 -0.32746357 -0.16823071 -0.26254135  0.07047758 -0.7042922
  0.1372576   0.06062786  0.06479052 -0.11237999 -0.03725365 -0.15076816
 -0.06765191  0.3483088   0.08438241  0.08398774  0.24307759 -0.18607536
  0.09483893 -0.39971793 -0.25020593 -0.19169647  0.37169236 -0.5526572
 -0.16141169 -0.09200899  0.03682638 -0.24559082  0.36410147  0.24568465
 -0.35796794  0.4000603   0.16157433 -0.38644806 -0.92824584  0.06200597
  0.21786208 -0.2899246   0.40019453  0.28365323  0.08914898 -0.0830568
  0.23344028 -0.01339766  0.11852799 -0.16985223  0.08948139 -0.2098016
 -0.10880087 -0.09219439  0.33692613 -0.28639394  0.0903092   

### Save results to a .pkl file ###

In [16]:
output_file = "tracks_embeddings_similars.pkl"
try:
    with open(output_file, 'wb') as f:
        pickle.dump(results, f)
    print(f"Embeddings saved to {output_file}")
except Exception as e:
    print(f"Error saving embeddings: {e}")

Embeddings saved to tracks_embeddings_similars.pkl


### Load the .pkl file ###

In [17]:
pkl_path = '/kaggle/input/tracks-embeddings-similars/tracks_embeddings_similars.pkl'
with open(pkl_path, 'rb') as file:
    tracks_embeddings_similars = pickle.load(file)

### Checking loaded file ###

In [18]:
print('Length of results: ', len(tracks_embeddings_similars))
print('Length of result for first song: ', len(tracks_embeddings_similars[0]))
print('The spotify_id of first song: ', tracks_embeddings_similars[0][0])
print('Embedding of first song: ', tracks_embeddings_similars[0][1])
print('Top N similar songs to first song: ', tracks_embeddings_similars[0][2])

Length of results:  1500
Length of result for first song:  3
The spotify_id of first song:  0flZwJevfk2DptAn9HMCry
Embedding of first song:  [-0.68818575  0.13197674  0.24371189 -0.32256    -0.06723172 -0.5038438
  0.16905946  0.1745734   0.02652138 -0.41278073  0.2817328  -0.3267785
 -0.25943887 -0.19831687 -0.17495061 -0.01648985  0.25906953  0.11717907
 -0.41598013 -0.32746357 -0.16823071 -0.26254135  0.07047758 -0.7042922
  0.1372576   0.06062786  0.06479052 -0.11237999 -0.03725365 -0.15076816
 -0.06765191  0.3483088   0.08438241  0.08398774  0.24307759 -0.18607536
  0.09483893 -0.39971793 -0.25020593 -0.19169647  0.37169236 -0.5526572
 -0.16141169 -0.09200899  0.03682638 -0.24559082  0.36410147  0.24568465
 -0.35796794  0.4000603   0.16157433 -0.38644806 -0.92824584  0.06200597
  0.21786208 -0.2899246   0.40019453  0.28365323  0.08914898 -0.0830568
  0.23344028 -0.01339766  0.11852799 -0.16985223  0.08948139 -0.2098016
 -0.10880087 -0.09219439  0.33692613 -0.28639394  0.0903092   

### Define a function to perform content based filtering based on given spotify_id ###

In [19]:
def content_based_filtering(spotify_id, tracks_folder, pkl_path, N = 15):
    # Load the .pkl file
    with open(pkl_path, 'rb') as file:
        tracks_embeddings_similars = pickle.load(file)
    
    # The spotify_id exists in tracks_embeddings_similars
    for track_spotify_id, _, similars in tracks_embeddings_similars:
        if track_spotify_id == spotify_id:
            print(f"Top {len(similars)} similar songs for {spotify_id}: {similars}")
            return similars
            
    # If spotify_id not found, process new track
    print(f"{spotify_id} not found in existing database. Processing new track...")
    # Load and preprocess new .mp3 audio file
    audio_file = f"{spotify_id}.mp3"
    if not os.path.exists(audio_file):
        print(f"Error: {audio_file} not found in current directory!")
        return
    file_path = os.path.join(tracks_folder, audio_file)
    audio = preprocess_audio(file_path)
    if audio is None:
        print("Audio is None!")
    
    # Extract embedding
    new_embedding = extract_vggish_embedding(audio)
    if new_embedding is None:
         print("Embedding is None!")

    # Compute cosine similarities with existing embeddings
    spotify_ids = [sid for sid, _, _ in tracks_embeddings_similars]
    existing_embeddings = np.array([emb for _, emb, _ in tracks_embeddings_similars])
    new_embedding = new_embedding.reshape(1, -1)  # Reshape for sklearn
    similarities = cosine_similarity(new_embedding, existing_embeddings)[0]
    
    # Get top N similar songs
    if len(similarities) < N:
        print(f"Warning: Only {len(similarities)} songs available, expected {N}!")
        N = len(similarities)
    
    top_n_indices = np.argsort(similarities)[-N:][::-1]  # Descending order
    similar_ids = [spotify_ids[idx] for idx in top_n_indices]
    
    # Print similar songs
    print(f"Top {N} similar songs for {spotify_id}: {similar_ids}")
    
    # Append new entry to tracks_embeddings_similars
    tracks_embeddings_similars.append((spotify_id, new_embedding, similar_ids))
    
    # Save updated data to existing .pkl file
    output_file = "tracks_embeddings_similars.pkl"
    try:
        with open(output_file, 'wb') as f:
            pickle.dump(tracks_embeddings_similars, f)
        print(f"Updated {output_file} with new entry for {spotify_id}.")
    except Exception as e:
        print(f"Error saving updated .pkl file: {e}!")

    return similar_ids

### Define a function to get tags based on given spotify_id ###

In [20]:
def get_tags_for_song(spotify_id, csv_path):
    try:
        df = pd.read_csv(csv_path)
        match = df[df['spotify_id'] == spotify_id]
        
        # Check if spotify_id exists
        if match.empty:
            print(f"No track found with spotify_id: {spotify_id}")
            return None
        
        # Get tags 
        tags = match['tags'].iloc[0]  # Get first match
        
        # If tags are a string (e.g., "pop,rock"), split into a list
        if isinstance(tags, str):
            tags = tags.split(',')  # Adjust delimiter if needed (e.g., ';')
            tags = [tag.strip() for tag in tags]  # Remove whitespace
        
        return tags
    
    except FileNotFoundError:
        print(f"CSV file not found at: {csv_path}")
        return None
    except KeyError as e:
        print(f"Column not found: {e}. Available columns: {df.columns.tolist()}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

### Testing conetent based filtering function ###

In [21]:
csv_path = '/kaggle/input/spotify-musics/music_info.csv'
tracks_folder = '/kaggle/input/spotify-musics/tracks/tracks'
pkl_path = '/kaggle/input/tracks-embeddings-similars/tracks_embeddings_similars.pkl'
random_idx = random.randint(0, len(tracks_embeddings_similars) - 1)
print(f'Random index: {random_idx}')
random_spotify_id = tracks_embeddings_similars[random_idx][0]
print(f'Random spotify_id: {random_spotify_id}')
top_N_similars = content_based_filtering(random_spotify_id, tracks_folder, pkl_path)
print()
print(f'Tags for {random_spotify_id}: {get_tags_for_song(random_spotify_id, csv_path)}')
for i, sid in enumerate(top_N_similars):
    print(f'{i + 1} - Tags for {sid}: {get_tags_for_song(sid, csv_path)}')

Random index: 964
Random spotify_id: 5N2ybHaC9tqFvYxGj70haX
Top 15 similar songs for 5N2ybHaC9tqFvYxGj70haX: ['5c78KDEM5G2jXSzPbJ2poU', '1BcuTkrmqT5mYrFvDeM9lZ', '07Kv362UKOrAnvqYYNVUVA', '0RLlGSUMlV1px4ASzHOeXU', '0R7wyVHD96jcX4BboDpAP4', '0tEjTLnTwbf1I7z3cow9qy', '0EE51zRGKUd9AVtyM9jmj5', '0in9wt5lji2nlC35JGpTNm', '1q2yciMI9xjFWL8ZdiRrEo', '0oh61ZeBfXUJOqWzGWlVqu', '4jG5Cfg9Hn5ReXTHaGEW38', '0anO2GzDqalXT8xcuSncpi', '0RJ6CvfsnjMK3v4KK4Y6fo', '0wuLm98dMgCHF6g5SpKY71', '0aYgF3SUk1AxDpIdv5oAxO']

Tags for 5N2ybHaC9tqFvYxGj70haX: ['country']
1 - Tags for 5c78KDEM5G2jXSzPbJ2poU: nan
2 - Tags for 1BcuTkrmqT5mYrFvDeM9lZ: ['folk', 'singer_songwriter', 'britpop']
3 - Tags for 07Kv362UKOrAnvqYYNVUVA: ['pop', 'male_vocalists', '00s']
4 - Tags for 0RLlGSUMlV1px4ASzHOeXU: ['folk', 'singer_songwriter', 'soul', 'blues', 'acoustic', 'guitar']
5 - Tags for 0R7wyVHD96jcX4BboDpAP4: ['indie', 'pop', 'female_vocalists', 'swedish']
6 - Tags for 0tEjTLnTwbf1I7z3cow9qy: ['folk', 'singer_songwriter', 'countr